In [1]:
import time, sys
sys.path.append('/media/carson/New Volume/Chloe/chloebot/env36/lib/python3.6/site-packages')
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import import_ipynb
from MoveData import Options, json2datatools, num_batches, nopeak_mask, create_masks
from Encoder import Encoder
from Decoder import Decoder
from LearningDynamics import CosineWithRestarts

importing Jupyter notebook from MoveData.ipynb
importing Jupyter notebook from Encoder.ipynb
importing Jupyter notebook from Elements.ipynb
importing Jupyter notebook from Decoder.ipynb
importing Jupyter notebook from LearningDynamics.ipynb


In [9]:
if torch.cuda.is_available():
    print("you have", torch.cuda.device_count(), "GPUs")
    device = torch.device("cuda:0")
else:
    print('no GPUs detected')
    device = torch.device("cpu")

opt = Options(batchsize=2, device=device, epochs=25, lr=0.01, 
              beam_width=3, max_len = 25, save_path = '../saved/weights/model_weights')

data_iter, infield, outfield, opt = json2datatools(path='../saved/pairs.json',opt=opt)

you have 2 GPUs


In [10]:
class Transformer(nn.Module):
    def __init__(self, in_vocab_size, out_vocab_size, emb_dim, n_layers, heads, dropout):
        super().__init__()
        self.encoder = Encoder(in_vocab_size, emb_dim, n_layers, heads, dropout)
        self.decoder = Decoder(out_vocab_size, emb_dim, n_layers, heads, dropout)
        self.out = nn.Linear(emb_dim, out_vocab_size)
    def forward(self, src_seq, trg_seq, src_mask, trg_mask):
        e_output = self.encoder(src_seq, src_mask)
        d_output = self.decoder(trg_seq, e_output, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [22]:
emb_dim, n_layers, heads, dropout = 32, 3, 8, 0.01 
chloe = Transformer(len(infield.vocab), len(outfield.vocab), emb_dim, n_layers, heads, dropout)
chloe.load_state_dict(torch.load(opt.save_path))
if opt.device == torch.device("cuda:0"):
    chloe =  chloe.cuda()

In [23]:
next(chloe.parameters()).is_cuda

True

In [6]:
optimizer = torch.optim.Adam(chloe.parameters(), lr=opt.lr, betas=(0.9, 0.98), eps=1e-9)
scheduler = CosineWithRestarts(optimizer, T_max=num_batches(data_iter))

In [13]:
batch = next(iter(data_iter))
src = batch.listen.transpose(0,1)
trg = batch.reply.transpose(0,1)
trg_input = trg[:, :-1]

In [30]:
print(infield.vocab.stoi)
print(" ------------------------------------------------------ ")
print('input_vocab_size =', len(infield.vocab), ', output_vocab_size =', len(outfield.vocab))
print(" ------------------------------------------------------ ")
print(src, src.shape)
print(" ------------------------------------------------------ ")
print(trg, trg.shape)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f5c2daea2e8>>, {'<unk>': 0, '<pad>': 1, '?': 2, 'you': 3, 'chloe': 4, 'i': 5, 'ok': 6, 'are': 7, 'hi': 8, 'a': 9, 'bye': 10, 'haha': 11, 'hello': 12, 'how': 13, 'later': 14, 'lol': 15, 'sure': 16, 'alive': 17, 'am': 18, 'any': 19, 'do': 20, 'dont': 21, 'dunno': 22, 'go': 23, 'gotta': 24, 'ill': 25, 'im': 26, 'joke': 27, 'know': 28, 'me': 29, 'more': 30, 'robot': 31, 'see': 32, 'talk': 33, 'tell': 34, 'think': 35, 'to': 36, 'true': 37, 'ttyl': 38, 'vicki': 39, 'what': 40, 'who': 41})
 ------------------------------------------------------ 
input_vocab_size= 42 , output_vocab_size= 66
 ------------------------------------------------------ 
tensor([[34, 29,  9, 27,  4,  1],
        [40, 20,  3, 35,  4,  2]], device='cuda:0') torch.Size([2, 6])
 ------------------------------------------------------ 
tensor([[ 2, 17, 14, 15, 13, 10, 18,  4,  5,  3,  1],
        [ 2, 51, 40, 25, 47,  6, 53, 58, 25, 52

The shape of the source mask is (batch_size, 1, input_sequence_length)
The shape of the target mask is (batch_size, output_sequence_length, output_sequence_length)

In [18]:
src_mask, trg_mask = create_masks(src, trg_input, opt)
print(src_mask, src_mask.shape)
print(" ------------------------------------------------------ ")
print(trg_mask, trg_mask.shape)

tensor([[[ True,  True,  True,  True,  True, False]],

        [[ True,  True,  True,  True,  True,  True]]], device='cuda:0') torch.Size([2, 1, 6])
 --------------------------------------- 
tensor([[[ True, False, False, False, False, False, False, False, False, False],
         [ True,  True, False, False, False, False, False, False, False, False],
         [ True,  True,  True, False, False, False, False, False, False, False],
         [ True,  True,  True,  True, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True, False, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True]]

The shape of the source mask is (batch_size, output_sequence_length, output_vocab_size)
For every reply in batch_size, chloe says output_sequence_length number of words, for each output position, 66 numbers are given 

In [31]:
preds = chloe(src, trg_input, src_mask, trg_mask)
print(preds, preds.shape)

tensor([[[-1.6977, -1.1294, -0.7040,  ..., -1.9627,  2.2752, -0.6955],
         [-0.5120, -1.4015, -2.1118,  ..., -0.5819, -3.7025, -1.4250],
         [-1.7567, -2.1324, -1.2924,  ..., -2.0275, -2.6420,  1.3241],
         ...,
         [-1.0377, -1.1146, -2.0184,  ...,  3.0106, -2.6459, -1.0744],
         [-3.4373, -3.3411, -2.9979,  ..., -5.4473, -1.8967, -3.1632],
         [-2.5257, -1.4176, -1.6269,  ..., -4.0867,  2.6793, -2.8859]],

        [[-1.2958, -1.0706, -0.5325,  ..., -0.1642,  1.6314, -0.8045],
         [ 0.6351, -0.6384,  0.3854,  ...,  2.7451, -1.7780,  1.9991],
         [-2.2120, -1.2582, -1.9899,  ..., -2.3293,  1.4207,  0.5253],
         ...,
         [-2.5363, -1.2725, -2.0955,  ..., -2.0993,  0.9256,  0.2223],
         [ 0.8198,  0.4271,  1.5342,  ...,  0.7719,  2.0581, -4.2233],
         [-2.9531, -2.4199, -2.5831,  ..., -4.2000, -2.0004, -3.3899]]],
       device='cuda:0', grad_fn=<AddBackward0>) torch.Size([2, 10, 66])


In [5]:
def trainer(model, data_iterator, options, optimizer, scheduler):

    if torch.cuda.is_available() and options.device == torch.device("cuda:0"):
        print("a GPU was detected, model will be trained on GPU")
        model = model.cuda()
    else:
        print("training on cpu")

    model.train()
    start = time.time()
    best_loss = 100
    for epoch in range(options.epochs):
        total_loss = 0
        for i, batch in enumerate(data_iterator): 
            src = batch.listen.transpose(0,1)
            trg = batch.reply.transpose(0,1)
            trg_input = trg[:, :-1]
            src_mask, trg_mask = create_masks(src, trg_input, options)
            preds = model(src, trg_input, src_mask, trg_mask)
            ys = trg[:, 1:].contiguous().view(-1)
            optimizer.zero_grad()
            batch_loss = F.cross_entropy(preds.view(-1, preds.size(-1)), 
                                         ys, ignore_index = options.trg_pad)
            batch_loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += batch_loss.item()

        epoch_loss = total_loss/(num_batches(data_iterator)+1)
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model.state_dict(), options.save_path)
        print("%dm: epoch %d loss = %.3f" %((time.time() - start)//60, epoch, epoch_loss))
        total_loss = 0

    return model